In [1]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap

from nltk.tokenize import WordPunctTokenizer
from matplotlib import pyplot as plt

from IPython.display import clear_output

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=762624147a8a67fa46cd6904808ba64b5fb9169b5d749fcef9a3bd3b52a66ecc
  Stored in directory: /root/.cache/pip/wheels/f4/3e/1c/596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=100878adc392ba197658e36d3706f8a9674c6d0dda9066c445ade41a3d4e

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
# выгружаем датасет:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

--2023-03-21 17:08:21--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2023-03-21 17:08:21--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf3261d02bef4a2bc3c84d5b61f.dl.dropboxusercontent.com/cd/0/get/B4r3XVLbhpvtGUZ0MGdp663wAbR-08rgaLa179yzxOFuI9YS9aUxl1J0joeeY4EuJxjUCmjqFByewUnhEXUYkzp8eRV0qTt-zuL0DIFrgpauIbjmP5Ok6edVIKf6ATb3em4yNdW6LlZOrMAmQEavxnxcPmLWwJuDEic9iz66Q_6Glg/file?dl=1# [following]
--2023-03-21 17:08:21--  https://ucf3261d02bef4a2bc3c84d5b61f.dl.dropboxusercontent.com/cd/0/get/B4r3XVLbhpvtGUZ0MGdp663wAbR-08rgaLa179yzxOFuI9YS9aUxl1J0joeeY4EuJxjUCmjqFByewUnhEXUYkzp

In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [4]:
tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [5]:
data_tok = []
for i in data:
  s= [x.lower() for x in tokenizer.tokenize(i)]
  data_tok.append(s)



#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
data_tok[0:5]

[['can',
  'i',
  'get',
  'back',
  'with',
  'my',
  'ex',
  'even',
  'though',
  'she',
  'is',
  'pregnant',
  'with',
  'another',
  'guy',
  "'",
  's',
  'baby',
  '?'],
 ['what',
  'are',
  'some',
  'ways',
  'to',
  'overcome',
  'a',
  'fast',
  'food',
  'addiction',
  '?'],
 ['who',
  'were',
  'the',
  'great',
  'chinese',
  'soldiers',
  'and',
  'leaders',
  'who',
  'fought',
  'in',
  'ww2',
  '?'],
 ['what', 'are', 'zip', 'codes', 'in', 'the', 'bay', 'area', '?'],
 ['why',
  'was',
  'george',
  'rr',
  'martin',
  'critical',
  'of',
  'jk',
  'rowling',
  'after',
  'losing',
  'the',
  'hugo',
  'award',
  '?']]

###Задание 2: Подсчитайте топ10 самых популярных лем в рамках data

In [7]:
from nltk.probability import FreqDist

In [8]:
all_tokens = []
for a in data_tok:
    all_tokens.extend(a)
fdist = FreqDist(all_tokens)
fdist.most_common(10)

[('?', 552413),
 ('the', 252068),
 ('what', 214798),
 ('is', 185392),
 ('a', 155726),
 ('i', 149735),
 ('to', 141788),
 ('in', 139787),
 ('how', 135687),
 ('of', 112001)]

###Задание 3: Подсчитайте количество разных слов до и после лемматизации

In [10]:
after_lemm = set()
for a in all_tokens:
    after_lemm.add(lemmatizer.lemmatize(a))

In [11]:
do = len(set(fdist))
posle = len(after_lemm)
print(f'Do lem = {do}, Posle lem = {posle}')

Do lem = 87820, Posle lem = 80304


###Задание 4: Подсчитайте количество разных слов до и после стемминга

In [12]:
after_lemm_stem = set()
for a in all_tokens:
    after_lemm_stem.add(ps.stem(a))

In [13]:
print(f'Do stem = {do}, Posle stem = {len(after_lemm_stem)}')

Do stem = 87820, Posle stem = 67026


###Задание 5: Подсчитайте количество разных слов
нужно посчитать (на базе предыдущих упражнений)
- до лемматизации и стеминнга
- после стемминга
- после лемматизации
- после стеминга и лемматизации

и отдельно сравнить (для важности корректного порядка процедур)

1. сначала стеминг, потом лемматизацию
2. сначала лемматизацию, потом стемминг

In [14]:
#до лемматизации и стеминнга
print(len(set(fdist)))

87820


In [15]:
#после стемминга
print(len(after_lemm_stem))

67026


In [16]:
#после лемматизации
print(posle)

80304


In [19]:
lemm_to_stem = set()
for i in all_tokens:
    lemm_to_stem.add(ps.stem(lemmatizer.lemmatize(i)))

In [20]:
print(f'Cначала лемматизация, потом стемминг = {len(lemm_to_stem)}')

Cначала лемматизация, потом стемминг = 66835


In [21]:
stem_to_lemm = set()
for i in all_tokens:
    stem_to_lemm.add(lemmatizer.lemmatize(ps.stem(i)))

In [22]:
print(f'Cначала стеминг, потом лемматизация = {len(stem_to_lemm)}')

Cначала стеминг, потом лемматизация = 66818


In [ ]:
#после выполнения задяния, можно сделать вывод, что сначала необходимо выполнять лемматизацию, а потом стемминг.